# AxiusMEM™ Quickstart

This notebook demonstrates how to get started with AxiusMEM™: loading the ontology, connecting to a triplestore, adding triples, and running queries.

In [3]:
# If running in Colab or a fresh environment, uncomment the following to install dependencies:
# !pip install axiusmem python-dotenv rdflib pandas requests

import os
from dotenv import load_dotenv
load_dotenv()

from axiusmem import AxiusMEM, get_default_ontology_path
print(os.getenv("TRIPLESTORE_URL"))
print(os.getenv("TRIPLESTORE_USER"))
print(os.getenv("TRIPLESTORE_REPOSITORY"))
print(os.getenv("TRIPLESTORE_TYPE"))


mem = AxiusMEM(ontology_path=get_default_ontology_path())




http://localhost:3030
admin
agents
jena


## Load Ontology and Initialize AxiusMEM™

In [5]:
# Path to your ontology file (adjust if needed)
ontology_path = get_default_ontology_path()
print(ontology_path)
mem = AxiusMEM(ontology_path=ontology_path)
print("Ontology loaded.")

C:\GitHub\AxiusMEM\src\axiusmem\axiusmem_ontology.ttl
Ontology loaded.


## Connect to Triplestore

In [6]:
from axiusmem import AxiusMEM
from axiusmem.adapters.base import get_triplestore_adapter_from_env

mem = AxiusMEM()
adapter = get_triplestore_adapter_from_env()
adapter.connect()  # Only needed for some adapters

# Now you can use adapter.sparql_select(), adapter.sparql_update(), etc.

jena


Check for the TRIPLESTORE REPOSITORY and load the ontology into it

In [7]:
from axiusmem.adapters.base import get_triplestore_adapter_from_env
from axiusmem import load_default_ontology
import os

repo = os.getenv("TRIPLESTORE_REPOSITORY")
adapter = get_triplestore_adapter_from_env()

def ensure_graphdb_repo_and_load_ontology(adapter, repo):
    resp = adapter.session.get(f"{adapter.url}/rest/repositories")
    repos = [r['id'] for r in resp.json()]
    if repo not in repos:
        config = {
            "id": repo,
            "title": repo,
            "type": "free",
            "params": {
                "repositoryType": "file-repository",
                "ruleset": "owl-horst-optimized",
                "storage-folder": f"/tmp/{repo}"
            }
        }
        r = adapter.session.post(f"{adapter.url}/rest/repositories", json=config)
        r.raise_for_status()
        print(f"Created GraphDB repository '{repo}'.")
    else:
        print(f"Repository '{repo}' already exists.")
    adapter.repository = repo
    load_default_ontology(adapter)
    print("Loaded ontology into repository.")

def ensure_jena_dataset_and_load_ontology(adapter, repo):
    datasets = adapter.list_datasets()
    dataset_names = [d['ds.name'] for d in datasets.get('datasets', [])]
    if repo not in dataset_names:
        try:
            created = adapter.create_dataset(repo, db_type="mem")
            if created:
                print(f"Created Jena dataset '{repo}'.")
            else:
                print(f"Failed to create Jena dataset '{repo}'.")
        except Exception as e:
            # If it's a 409 Conflict, treat as already exists
            if hasattr(e, 'response') and getattr(e.response, 'status_code', None) == 409:
                print(f"Dataset '{repo}' already exists (409 Conflict).")
            else:
                raise
    else:
        print(f"Dataset '{repo}' already exists.")
    adapter.dataset = repo
    load_default_ontology(adapter)
    print("Loaded ontology into dataset.")

if adapter.__class__.__name__ == "GraphDBAdapter":
    ensure_graphdb_repo_and_load_ontology(adapter, repo)
elif adapter.__class__.__name__ == "JenaAdapter":
    ensure_jena_dataset_and_load_ontology(adapter, repo)
else:
    print("Automatic repository/dataset creation not implemented for this adapter.")

jena
Dataset 'agents' already exists (409 Conflict).
Loaded ontology into dataset.


## Add Example Triples

In [9]:
from rdflib import URIRef, Literal

# Example triple: Alice knows Bob
s = URIRef("https://axius.info/axiusmem/alice")
p = URIRef("http://xmlns.com/foaf/0.1/knows")
o = URIRef("https://axius.info/axiusmem/bob")

mem.add_triples([(s, p, o)])
print("Added triple: Alice knows Bob")

Added triple: Alice knows Bob


## Query the Knowledge Graph

In [10]:
query = """
SELECT ?s ?o WHERE {
  ?s <http://xmlns.com/foaf/0.1/knows> ?o .
}
"""
results, df = mem.select(query)
print("Query results:")
print(df)

Query results:
                                   s                                o
0           http://example.org/alice           http://example.org/bob
1  https://axius.info/axiusmem/alice  https://axius.info/axiusmem/bob


## (Optional) Add Valid Time to a Triple

In [11]:
from axiusmem.temporal import add_valid_time

# Add a triple with valid time
mem.add_triples([(s, p, o)], valid_time={"from": "2024-01-01", "to": "2024-12-31"})
print("Added triple with valid time.")

Added triple with valid time.


## (Optional) Query with Temporal Reasoning

In [12]:
from axiusmem.temporal import query_point_in_time

# Query for triples valid at a specific time
subgraph = query_point_in_time(mem.graph, "2024-06-01")
print("Triples valid on 2024-06-01:")
for stmt in subgraph:
    print(stmt)

Triples valid on 2024-06-01:
(rdflib.term.URIRef('https://axius.info/axiusmem/alice'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/knows'), rdflib.term.URIRef('https://axius.info/axiusmem/bob'))
